In [ ]:
!pip install langchain unstructured openai tiktoken

In [ ]:
import os
import nltk
import json
import magic
import random
import datetime
import pandas as pd
from datetime import timedelta
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

!rm -rf './data'
os.makedirs('data')

os.environ["OPENAI_API_KEY"] = "HERE IS THE API KEY"

def genRandomDate():
  end_date = datetime.datetime.now()
  start_date = end_date - timedelta(days=1000)
  return str(start_date + (end_date - start_date) * random.random())

def genRandomUser():
  users = ["Poppyhead", "Zygomatic", "Calathus", "Skedaddle", "Negative", "Rejigger", "Apolaustic", "Phalanx", "Havelock", "Sardoodledom", "Galimatias", "Palomino", "Thaumatogeny", "Piffling", "Vaniloquence", "Trencherman", "Pilgarlic", "Curlicue", "KilimZol", "EnchanMizzen", "Ludicrism", "DartmoorYawp", "Anorchous", "Umpteen", "IquidDuck", "JcavalJunket", "Symptosis", "RuffminOxter", "Luciform", "Antimacassar", "Onomatopoeia", "Blackguard", "Xerotripsis", "Orgulous", "Almacantar", "Comeuppance", "ChichimWonky", "Logorrhea", "Knickknack", "Cornucopia", "Rupellary", "Peewee", "Armigerous", "Gardyloo", "Nephralgia", "Piccadilly", "Aleeman171999", "Harebrained", "Nektonseph2004", "Limburger", "Zonulet", "Sambur", "Aasvogel", "Primp", "Cinemuck", "Jubilee", "Capripede", "Flophouse", "Tachyphrasia", "Ephemeral", "Paraenesis", "Normalcy", "Morwolf1596", "Hotbed", "Limnguinhd", "Parapet", "Ateknia", "Taradiddle", "Blowback", "Kudzu", "Opsiometer", "Outflow", "Prismatic", "Roundabout", "Bellycheer", "Ewer", "Rabblefalnd", "Mufti", "Anemograph", "Cheeky"]
  return random.choice(users)


In [ ]:
data = pd.read_csv("test.csv", delimiter=",")

with open(f'data/file.jsonl', 'w', encoding='utf-8') as f:
  for row in data['review']:
    f.write(json.dumps(str({
        'username': genRandomUser(),
        'commented_on': genRandomDate(),
        'comment': row.split('READ MORE')[0]
    })) + '\n')

In [ ]:
# load all documents
loader = DirectoryLoader('data', glob="**/*.jsonl")
documents = loader.load()

# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 0,
    length_function = len,
)

texts = text_splitter.split_documents(documents)

# initialize the Summary chain
chain = load_summarize_chain(llm=ChatOpenAI(), chain_type='refine')

In [ ]:
# texts[:5]

In [ ]:
response = chain.run(texts[:5])
display(Markdown(f"Ans: <b>{response}</b>"))

# while True: 
#   query = input("Ques: ")
#   if query == "exit": break

#   response = chain({"input_documents": texts[:5], "question": query}, return_only_outputs=True)
#   display(Markdown(f"Ans: <b>{response['output_text']}</b>"))
#   # print('source_documents', response['source_documents'])

Ans: <b>Five users have praised the powerful bass and good sound quality of a product that has outstanding build quality made of good quality plastic. The product is also portable, with a long battery life, and considered value for money. However, some users noted discomfort from prolonged use due to its small size, and one recommended having a wired alternative for when the battery runs low. The Bluetooth connectivity is average, and the calling feature is poor. Some users reported that the sound for alerts, notifications, and calls is excessively loud and irritating. One user reported that the microphone doesn't work well over Bluetooth, making it difficult for the other person to hear the user's speech clearly. Additionally, despite some positive reviews on battery life, one user found the microphone was not clear when using the product while driving. Another user reported that the bass is not as heavy as advertised, and also noted issues with the Bluetooth connectivity when playing certain music tracks. Nonetheless, the product remains a recommended option for those seeking a budget-friendly option for music consumption.</b>